In [1]:
from bs4 import BeautifulSoup
import requests
from multiprocessing import Pool

GIST_URL = "https://gist.github.com/paulmillr/2657075"

def get_soup_from_url(u):
    return BeautifulSoup(requests.get(u).content, 'html.parser')

In [9]:
def get_all_usernames(url):
    soup = get_soup_from_url(url)
    rows = soup.select('article table tbody tr')
    usernames = [row.find('a').text for row in rows]
    return usernames

In [10]:
def get_user_mean_stars(username):
    print(f'Retrieving infos for user {username}...')
    url = f'https://api.github.com/users/{username}/repos'
    headers = {'Authorization': 'token XXXXXXXXXXXXX'}
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print('oops')
        print(resp.content)
    repos = resp.json()
    nb_repos = len(repos)
    if nb_repos == 0:
        return 0
    total_stars = sum(repo['stargazers_count'] for repo in repos)
    return total_stars / nb_repos

In [11]:
def get_user_infos(username):
    return {
        'name': username,
        'mean_stars': get_user_mean_stars(username),
    }

def top_users():
    all_usernames = get_all_usernames(GIST_URL)
    scores = [get_user_infos(u) for u in all_usernames]
    return scores